In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import joblib

try:
    import xgboost as xgb
    has_xgb = True
except Exception as e:
    has_xgb = False
    

plt.rcParams['figure.figsize'] = (8,5)


In [2]:
# Cell 2: ডেটা লোড (তোমার ফাইলের নাম/পাথ লিখবে)
# ধরে নিচ্ছি ফাইলগুলো: sales.csv, customers.csv, books.csv, transactions.csv
# যদি আলাদা নাম থাকে তবে এখানে বদল করো।
file1 = "All Book List.xlsx"
file2 = "Sell- 2023.xlsx"
file3 = "Sell - 2024.xlsx"
file4 = "Sell - 2025.xlsx"

df1 = pd.read_excel(file1)        # sales/orders
df2 = pd.read_excel(file2)        # customer info
df3 = pd.read_excel(file3)        # book info (title, genre, price)
df4 = pd.read_excel(file4)        # transactions (if any) or inventory
# Quick view
print("Shapes:", df1.shape, df2.shape, df3.shape, df4.shape)
df1.head()


Shapes: (3488, 11) (3080, 24) (5019, 25) (5316, 27)


,Category Of Books,Name of Books,Book's Genre,Publication,Discount %,Discounted Price,মুদ্রিত মুল্য,ক্রয় মুল্য,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,গ্রাফিক নভেল আমার দেখা নয়াচীন,NaN,সেন্টার ফর রিসার্চ এন্ড ইনফরমেশন,NaN,200.0,1000,NaN,NaN,NaN,NaN
1,বইনগর,সেই সব চরিত্র বুকমার্ক সেট,NaN,NaN,NaN,150.0,NaN,100.0,NaN,NaN,NaN
2,NaN,জীবনানন্দ দাশ ভিন্টেজ বুকমার্ক বক্সসেট (২০ পিস),NaN,NaN,NaN,200.0,NaN,150.0,NaN,NaN,NaN
3,ওরা হৃদয়ের রং চেনে না,Animal City (ম্যাও 🐱) বুকমার্ক সেট,NaN,NaN,NaN,120.0,NaN,70.0,NaN,NaN,NaN
4,NaN,বুক-রিডার বুকমার্ক সেট (৮ পিস),NaN,NaN,NaN,40.0,NaN,25.0,NaN,NaN,NaN


In [3]:
df2.head()

,Unnamed: 0,SL,Gender,Age,Repeated Customer?,Name of Books,Unit,Review(Out of 5),Confirmation,Price,...,Sell,Delivery Method,Delivery Status,Address,Customer Phone,Unnamed: 19,MONTHLY ONLINE SALES,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,1.0,1.0,Female,41.0,Yes,এরা কারা,1.0,3.8,Confirmed,170.0,...,210.0,Courier (Office),Delivered,"সোনা মিয়া মার্কেট, আদমজীনগর, সিদ্ধিরগঞ্জ, নারা...",1538318047,NaN,NaN,NaN,NaN,Profit Count (14.28%)
1,1.0,2.0,Male,44.0,No,ছোটদের স্পোকেন ইংলিশ,1.0,4.5,Confirmed,168.0,...,168.0,In Person,Delivered,ঢাকা বিশ্ববিদ্যালয় ক্যাম্পাস,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,3.0,Female,46.0,No,ছায়া নট,1.0,3.5,Confirmed,108.0,...,413.0,Courier (Office),Delivered,"বাবুবাজার, ঢাকা",NaN,NaN,NaN,NaN,NaN,1937.6532
3,NaN,NaN,NaN,NaN,NaN,অগ্নিবীণা,1.0,NaN,NaN,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1170.246
4,NaN,NaN,NaN,NaN,NaN,গাভী বৃত্তান্ত,1.0,NaN,NaN,180.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3107.8992


In [4]:
df3.head()

,Unnamed: 0,SL,Gender,Age,Review(Out of 5),Name of Books,Unit,Repeated Customers?,Confirmation,Price,...,Delivery Method,Delivery Status,Address,Customer Phone,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,1.0,610,Female,30.0,4.5,দারবিশ (পেপারব্যাক),1.0,No,Confirmed,199.0,...,Courier (Office),Shipped,"Ranishonkoil,Thakurgaon",Md.Raiyan Islam Rokib\n01316852006,NaN,NaN,NaN,NaN,NaN,Profit Count (14.28%)
1,NaN,NaN,NaN,NaN,0.0,ফ্রন্টলাইন (পেপারব্যাক),1.0,NaN,NaN,199.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,0.0,নূর (পেপারব্যাক),1.0,NaN,NaN,199.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.0,অন্তিম (পেপারব্যাক),1.0,NaN,NaN,199.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9265.7208
4,NaN,NaN,NaN,NaN,0.0,রাখাল (পেপারব্যাক),1.0,NaN,NaN,199.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16595.9304


In [5]:
df4.head()

,Unnamed: 0,SL,Gender,Age,Repeated Customers?,Title,Unit,Review(Out of 5),Confirmation,Price,...,Address,Customer Phone,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,1.0,1679,Female,58.0,Yes,আরশের মেহমান,1.0,4.5,Confirmed,100.0,...,"Mathergonj, Jamalpur",Lafi Hasan \n01400294018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Profit Count (14.28%)
1,NaN,NaN,NaN,NaN,NaN,বিশ্বাসের সন্ধানে,1.0,NaN,NaN,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,নানা রঙের জীবন,1.0,NaN,NaN,165.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,হারিয়ে যাওয়া সম্পদ,1.0,NaN,NaN,215.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,মুসআব ইবনু উমাইর,1.0,NaN,NaN,65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Step 1️⃣: প্রয়োজনীয় লাইব্রেরি ইমপোর্ট
import pandas as pd
import glob

# Step 2️⃣: ফোল্ডারে থাকা সব Excel ফাইল খুঁজে বের করা
# যদি ফাইলগুলো একই ফোল্ডারে থাকে, তাহলে শুধু "*.xlsx" লিখলেই হবে
files = glob.glob("*.xlsx")    

print("পাওয়া ফাইলগুলো:", files)

# Step 3️⃣: প্রতিটা ফাইল আলাদা করে পড়া
df_list = []
for file in files:
    df = pd.read_excel(file)
    df['source_file'] = file   # চাইলে ফাইল নামও রাখে দিতে পারো (কোন মাসের ডেটা বোঝার জন্য)
    df_list.append(df)

# Step 4️⃣: সব DataFrame একসাথে merge (concatenate)
merged_df = pd.concat(df_list, ignore_index=True)

# Step 5️⃣: ফলাফল দেখা
print("Merged shape:", merged_df.shape)
print(merged_df.head())

# Step 6️⃣: ডুপ্লিকেট রো মুছে ফেলা (যদি থাকে)
merged_df.drop_duplicates(inplace=True)

# Step 7️⃣: Missing value থাকলে পূরণ করা
merged_df.fillna(0, inplace=True)

# Step 8️⃣: চাইলে নতুন Excel ফাইলে সেভ করে রাখো
merged_df.to_excel("merged_sales_data.xlsx", index=False)

print("✅ সব ফাইল merge হয়ে গেছে! saved as merged_sales_data.xlsx")


পাওয়া ফাইলগুলো: ['All Book List.xlsx', 'merged_sales_data.xlsx', 'Sell - 2024.xlsx', 'Sell - 2025.xlsx', 'Sell- 2023.xlsx']
Merged shape: (37374, 41)
       Category Of Books                                    Name of Books  \
0                    NaN                   গ্রাফিক নভেল আমার দেখা নয়াচীন    
1                  বইনগর                       সেই সব চরিত্র বুকমার্ক সেট   
2                    NaN  জীবনানন্দ দাশ ভিন্টেজ বুকমার্ক বক্সসেট (২০ পিস)   
3  ওরা হৃদয়ের রং চেনে না              Animal City (ম্যাও 🐱)  বুকমার্ক সেট   
4                    NaN                   বুক-রিডার বুকমার্ক সেট (৮ পিস)   

   Book's Genre                        Publication Discount %  \
0           NaN  সেন্টার ফর রিসার্চ এন্ড ইনফরমেশন         NaN   
1           NaN                                NaN        NaN   
2           NaN                                NaN        NaN   
3           NaN                                NaN        NaN   
4           NaN                                NaN        NaN 

In [7]:
merged_df.head()


,Category Of Books,Name of Books,Book's Genre,Publication,Discount %,Discounted Price,মুদ্রিত মুল্য,ক্রয় মুল্য,Unnamed: 9,Unnamed: 10,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Title,Unnamed: 25,Unnamed: 26,Repeated Customer?,MONTHLY ONLINE SALES
0,0,গ্রাফিক নভেল আমার দেখা নয়াচীন,0.0,সেন্টার ফর রিসার্চ এন্ড ইনফরমেশন,0,200.0,1000,0.0,0,0,...,0,0,0.0,0,0,0,0.0,0,0,0.0
1,বইনগর,সেই সব চরিত্র বুকমার্ক সেট,0.0,0,0,150.0,0,100.0,0,0,...,0,0,0.0,0,0,0,0.0,0,0,0.0
2,0,জীবনানন্দ দাশ ভিন্টেজ বুকমার্ক বক্সসেট (২০ পিস),0.0,0,0,200.0,0,150.0,0,0,...,0,0,0.0,0,0,0,0.0,0,0,0.0
3,ওরা হৃদয়ের রং চেনে না,Animal City (ম্যাও 🐱) বুকমার্ক সেট,0.0,0,0,120.0,0,70.0,0,0,...,0,0,0.0,0,0,0,0.0,0,0,0.0
4,0,বুক-রিডার বুকমার্ক সেট (৮ পিস),0.0,0,0,40.0,0,25.0,0,0,...,0,0,0.0,0,0,0,0.0,0,0,0.0


In [8]:
merged_df.columns

Index(['Category Of Books', 'Name of Books', 'Book's Genre', 'Publication',
       'Discount %', 'Discounted Price', 'মুদ্রিত মুল্য', ' ক্রয় মুল্য',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'source_file', 'Unnamed: 0',
       'SL', 'Gender', 'Age', 'Review(Out of 5)', 'Unit',
       'Repeated Customers?', 'Confirmation', 'Price', 'Cost',
       'Payment Method', 'Status', 'Order Date', 'Sell', 'Delivery Method',
       'Delivery Status', 'Address', 'Customer Phone', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Title', 'Unnamed: 25', 'Unnamed: 26',
       'Repeated Customer?', 'MONTHLY ONLINE SALES'],
      dtype='object')

In [9]:
# Cell 4: ডেটা টাইপ, missing value summary, basic cleaning
print(df.info())
print("\nMissing values per column:\n", df.isna().sum())

# উদাহরণ: date টাইপ কনভার্ট
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

# উদাহরণ: numeric conversion
for col in ['quantity','price','discount','total_amount']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# কনডিশনাল: total_amount না থাকলে হিসাব
if 'total_amount' not in df.columns and ('quantity' in df.columns and 'price' in df.columns):
    df['total_amount'] = df['quantity'] * df['price']


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3080 entries, 0 to 3079
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Unnamed: 0            1942 non-null   float64       
 1   SL                    608 non-null    float64       
 2   Gender                608 non-null    object        
 3   Age                   608 non-null    float64       
 4   Repeated Customer?    608 non-null    object        
 5   Name of Books         1744 non-null   object        
 6   Unit                  1743 non-null   float64       
 7   Review(Out of 5)      608 non-null    float64       
 8   Confirmation          608 non-null    object        
 9   Price                 1722 non-null   float64       
 10  Cost                  600 non-null    float64       
 11  Payment Method        607 non-null    object        
 12  Status                607 non-null    object        
 13  Order Date        

In [10]:
# Step 3: Drop unnecessary columns but keep 'Repeated Customers?'
cols_to_drop = [
    'Unnamed: 0', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
    'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
    'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26'
]
df = merged_df.drop(columns=cols_to_drop, errors='ignore').copy()

# Step 4: Clean column names (remove spaces and quotes, replace spaces with underscore)
df.columns = df.columns.str.strip().str.replace("'", "", regex=False).str.replace(" ", "_", regex=False)

# Check columns after cleaning
print("Columns after cleaning:")
print(list(df.columns))


Columns after cleaning:
['Category_Of_Books', 'Name_of_Books', 'Books_Genre', 'Publication', 'Discount_%', 'Discounted_Price', 'মুদ্রিত_মুল্য', 'ক্রয়_মুল্য', 'source_file', 'SL', 'Gender', 'Age', 'Review(Out_of_5)', 'Unit', 'Repeated_Customers?', 'Confirmation', 'Price', 'Cost', 'Payment_Method', 'Status', 'Order_Date', 'Sell', 'Delivery_Method', 'Delivery_Status', 'Address', 'Customer_Phone', 'Title', 'Repeated_Customer?', 'MONTHLY_ONLINE_SALES']


In [11]:
# Step 5: Convert numeric columns
maybe_numeric = ['Discount_%','Discounted_Price','মুদ্রিত_মূল্য','ক্রয়_মূল্য','Price','Cost','Sell','MONTHLY_ONLINE_SALES','Age','Review(Out_of_5)']
maybe_numeric = [c for c in maybe_numeric if c in df.columns]
for col in maybe_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Step 6: Fill missing values
num_cols = df.select_dtypes(include=['number']).columns.tolist()
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()

# Numeric: fill with median
for c in num_cols:
    df[c].fillna(df[c].median(), inplace=True)

# Categorical: fill with mode
for c in cat_cols:
    if df[c].isna().any():
        try:
            df[c].fillna(df[c].mode()[0], inplace=True)
        except:
            df[c].fillna("unknown", inplace=True)


In [12]:
# Step 7: Standardize Gender column
if 'Gender' in df.columns:
    df['Gender'] = df['Gender'].astype(str).str.strip().str.lower().map({'male':'male','female':'female'}).fillna('unknown')


In [13]:
# Step 8: Create Profit and High_Value_Customer flag
if 'Sell' in df.columns and 'Cost' in df.columns:
    df['Profit'] = df['Sell'] - df['Cost']
    threshold = df['Profit'].quantile(0.80)
    df['High_Value_Customer'] = (df['Profit'] >= threshold).astype(int)
    print("Profit and High_Value_Customer created. Threshold:", threshold)


Profit and High_Value_Customer created. Threshold: 300.0


In [14]:
# Step 9: Normalize 'Repeated_Customers?' to 0/1
if 'Repeated_Customers?' in df.columns:
    df['Repeated_Customers?'] = df['Repeated_Customers?'].astype(str).str.strip().str.lower()
    df['Repeated_Customers?'] = df['Repeated_Customers?'].replace(
        {'yes':1,'y':1,'true':1,'1':1,'no':0,'n':0,'false':0,'0':0}).fillna(0).astype(int)


In [15]:
# Step 10: Remove outliers using IQR method
outlier_cols = [c for c in ['Price','Sell','Cost','Profit','MONTHLY_ONLINE_SALES'] if c in df.columns]
for col in outlier_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    before = df.shape[0]
    df = df[(df[col] >= lower) & (df[col] <= upper)]
    after = df.shape[0]
    print(f"Outlier filter on {col}: removed {before-after} rows")


Outlier filter on Price: removed 740 rows
Outlier filter on Sell: removed 3562 rows
Outlier filter on Cost: removed 1492 rows
Outlier filter on Profit: removed 98 rows
Outlier filter on MONTHLY_ONLINE_SALES: removed 0 rows


In [16]:
# Step 11: Label encode small-cardinality categorical columns
le = LabelEncoder()
for c in ['Gender','Payment_Method','Delivery_Method','Delivery_Status']:
    if c in df.columns:
        df[c] = le.fit_transform(df[c].astype(str))


In [17]:
# Step 12: Scale numeric columns
scale_cols = [c for c in ['Price','Cost','Sell','Profit','MONTHLY_ONLINE_SALES','Age','Discount_%','Discounted_Price'] if c in df.columns]
if scale_cols:
    scaler = StandardScaler()
    df[scale_cols] = scaler.fit_transform(df[scale_cols])
    print("Scaled columns:", scale_cols)


Scaled columns: ['Price', 'Cost', 'Sell', 'Profit', 'MONTHLY_ONLINE_SALES', 'Age', 'Discount_%', 'Discounted_Price']


In [18]:
# Step 13: Split for ML if High_Value_Customer exists
if 'High_Value_Customer' in df.columns:
    X = df.drop(columns=['High_Value_Customer'], errors='ignore')
    y = df['High_Value_Customer']

    # Drop non-feature columns
    drop_for_X = [c for c in ['source_file','Customer_Phone','Address'] if c in X.columns]
    X = X.drop(columns=drop_for_X, errors='ignore')

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print("Train/Test split done. X_train shape:", X_train.shape, "X_test shape:", X_test.shape)


Train/Test split done. X_train shape: (11663, 27) X_test shape: (2916, 27)


In [19]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [20]:
from xgboost import XGBClassifier


In [21]:
# Check numeric columns for non-numeric values
numeric_cols = ['Price','Cost','Sell','Profit','MONTHLY_ONLINE_SALES','Age','Discount_%','Discounted_Price']  # adjust according to your df
for col in numeric_cols:
    if col in df.columns:
        non_numeric = df[~df[col].apply(lambda x: isinstance(x,(int,float,np.number)))]
        if not non_numeric.empty:
            print(f"Non-numeric values in {col}:")
            print(non_numeric[col].unique())


In [22]:
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # non-numeric values become NaN


In [23]:
for col in numeric_cols:
    if col in df.columns:
        df[col].fillna(df[col].median(), inplace=True)  # median fill


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"{name} Accuracy: {acc:.4f}")
    # Optional: confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title(f"{name} Confusion Matrix")
    plt.show()

# Compare all models
print("All model accuracies:")
for k,v in results.items():
    print(f"{k}: {v:.4f}")


ValueError: could not convert string to float: 'নূর'

In [27]:
from sklearn.neighbors import KNeighborsRegressor


In [29]:
# Target: Sell
from xgboost import XGBRegressor

if 'Sell' in df.columns:
    X_reg = df.drop(columns=['Sell','High_Value_Customer'], errors='ignore')
    y_reg = df['Sell']

    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
        X_reg, y_reg, test_size=0.2, random_state=42
    )

    reg_models = {
        'LinearRegression': LinearRegression(),
        'DecisionTreeRegressor': DecisionTreeRegressor(),
        'RandomForestRegressor': RandomForestRegressor(),
        'KNNRegressor': KNeighborsRegressor(),
        'XGBoostRegressor': XGBRegressor(objective='reg:squarederror')
    }

    results_reg = {}
    for name, model in reg_models.items():
        model.fit(X_train_reg, y_train_reg)
        y_pred = model.predict(X_test_reg)
        r2 = r2_score(y_test_reg, y_pred)
        rmse = mean_squared_error(y_test_reg, y_pred, squared=False)
        results_reg[name] = (r2, rmse)
        print(f"{name} - R2: {r2:.4f}, RMSE: {rmse:.4f}")


ValueError: could not convert string to float: 'সন্দীপন প্রকাশন'